In [1]:
from dataclasses import dataclass
import itertools
import logging
import random
import math
import numpy as np
import json
import pickle
import time
import torch
import sys
import yaml
import seaborn as sns
import matplotlib.pyplot as plt
from probe_utils import *

from omegaconf import OmegaConf
from pathlib import Path
from torch import nn, Tensor
from torch.nn import functional as F
from typing import List, Optional, Tuple
import os
# os.chdir("/data/tianyu_guo/birth")
from data import DataArgs, Dataset, iterate_batches, make_dataset
from ihead_full_model import ModelArgs, Transformer, forward_hook, test_value, test_sink

%load_ext autoreload
%autoreload 2


In [2]:
torch.cuda.set_device(3)

In [20]:
run_path_server = "/data/tianyu/birth/gens/final/dormant_copy"
# run_path_server2="/data/tianyu_guo/birth/gens/special/dormant_copy_2"
model, cfg, x, y, ds = load_model(run_path_local="/Users/guotianyu/GitHub/birth/gens/special/markov", run_path_server=run_path_server, n_layers=1, n_heads=1, bos_num=1, train_steps=10000, delim=0, with_data=True, data_path_local="/Users/guotianyu/GitHub/birth/data", data_path_server="/data/tianyu/birth/data")

In [21]:
rng = np.random.default_rng(27)
permute_idxs = [rng.permutation(range(1, x.shape[1])).tolist() for _ in range(x.shape[0])]
permute_idxs = [[0] + idxs for idxs in permute_idxs]
permute_x = torch.gather(x, 1, torch.tensor(permute_idxs).to(x.device))
probs = get_oracle_predicts(x, ds)
permute_probs = get_oracle_predicts(permute_x, ds)

predicts = model(x)
permute_predicts = model(permute_x)

In [23]:
risk = get_risk(probs, predicts, predict_in_logits=True)
risk_oracle = get_risk(probs, probs, predict_in_logits=False)
risk_permute = get_risk(permute_probs, permute_predicts, predict_in_logits=True)
risk_permute_oracle = get_risk(permute_probs, permute_probs, predict_in_logits=False)
print(risk[:,1:].mean(), risk_oracle[:,1:].mean(), risk_permute[:,1:].mean(), risk_permute_oracle[:,1:].mean())

tensor(1.6431, grad_fn=<MeanBackward0>) tensor(1.6420) tensor(2.3324, grad_fn=<MeanBackward0>) tensor(1.6420)


# whether positional encoding interferes with the OOD task?

In [28]:
no_bos_run_path_server = "/data/tianyu/birth/gens/final/dormant_copy_k3_bos0"
# run_path_server2="/data/tianyu_guo/birth/gens/special/dormant_copy_2"
model_no_bos, cfg_no_bos, _, _, _ = load_model(run_path_local="/Users/guotianyu/GitHub/birth/gens/special/markov", run_path_server=no_bos_run_path_server, n_layers=1, n_heads=1, bos_num=0, train_steps=10000, delim=0, with_data=True, data_path_local="/Users/guotianyu/GitHub/birth/data", data_path_server="/data/tianyu/birth/data")

In [30]:
predicts_no_bos = model(x[:, 1:])
permute_predicts_no_bos = model(permute_x[:, 1:])
probs_no_bos = probs[:, 1:, :]
permute_probs_no_bos = permute_probs[:, 1:, :]

risk_no_bos = get_risk(probs_no_bos, predicts_no_bos, predict_in_logits=True)
risk_oracle_no_bos = get_risk(probs_no_bos, probs_no_bos, predict_in_logits=False)
risk_permute_no_bos = get_risk(permute_probs_no_bos, permute_predicts_no_bos, predict_in_logits=True)
risk_permute_oracle_no_bos = get_risk(permute_probs_no_bos, permute_probs_no_bos, predict_in_logits=False)
print(risk_no_bos[:,1:].mean(), risk_oracle_no_bos[:,1:].mean(), risk_permute_no_bos[:,1:].mean(), risk_permute_oracle_no_bos[:,1:].mean())

tensor(1.6651, grad_fn=<MeanBackward0>) tensor(1.6394) tensor(2.3513, grad_fn=<MeanBackward0>) tensor(1.6419)
